<a href="https://colab.research.google.com/github/VanessaLaraPerez/TeoriaJuegos_PodadoAlfa-Beta/blob/main/Teoria_de_Juegos_e_Inteligencia_Artificial_VanessaLara.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
!pip install nashpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
import nashpy as nash
import numpy as np

Definimos las matrices de recompensas:

In [42]:
A = np.array([[-8, 0], [-10, -1]]) # A es el jugador filas.
B = np.array([[-8, -10], [0, -1]]) # B es el jugador columnas.
juego1 = nash.Game(A,B)
juego1

Bi matrix game with payoff matrices:

Row player:
[[ -8   0]
 [-10  -1]]

Column player:
[[ -8 -10]
 [  0  -1]]

Encontramos las estrategias de Equilibrio:

In [43]:
equilibrios = juego1.support_enumeration()
for eq in equilibrios:
    print(eq)

(array([1., 0.]), array([1., 0.]))


De manera similar, para un juego mas complejo con estrategias dominantes:

In [44]:
A2 = np.array([[6, 4, 4, 3],
               [5, 6, 0, 5],
               [5, 3, 6, 4],
               [2, 2, 3, 6]]) # A es el jugador filas.
               
B2 = np.array([[3, 4, 1, 0],
               [4, 3, 2, 1],
               [0, 2, 1, 4],
               [0, 3, 3, 1]]) # B es el jugador columnas.
juego2 = nash.Game(A2, B2)
juego2

Bi matrix game with payoff matrices:

Row player:
[[6 4 4 3]
 [5 6 0 5]
 [5 3 6 4]
 [2 2 3 6]]

Column player:
[[3 4 1 0]
 [4 3 2 1]
 [0 2 1 4]
 [0 3 3 1]]

Las estrategias resultantes son aquellas que no son dominadas:

In [45]:
equilibrios = juego2.support_enumeration()
for eq in equilibrios:
    print(eq)

(array([0.5, 0.5, 0. , 0. ]), array([0.66666667, 0.33333333, 0.        , 0.        ]))


# Minimax para 3 en Raya

In [46]:
import time

class Juego:
    def __init__(self):
        self.iniciar_juego()

    def iniciar_juego(self):
        self.estado_actual = [['.','.','.'],
                              ['.','.','.'],
                              ['.','.','.']]

        # X siempre juega primero:
        self.turno_jugador = 'X'

    def dibujar_tablero(self):
        for i in range(0, 3):
            for j in range(0, 3):
                print('{}|'.format(self.estado_actual[i][j]), end=" ")
            print()
        print()

    def es_valido(self, px, py):
      if px < 0 or px > 2 or py < 0 or py > 2:
          return False
      elif self.estado_actual[px][py] != '.':
          return False
      else:
          return True
    
    def es_fin(self):
      # Victoria en vertical:
      for i in range(0, 3):
          if (self.estado_actual[0][i] != '.' and
              self.estado_actual[0][i] == self.estado_actual[1][i] and
              self.estado_actual[1][i] == self.estado_actual[2][i]):
              return self.estado_actual[0][i]

      # Victoria en Horizontal:
      for i in range(0, 3):
          if (self.estado_actual[i] == ['X', 'X', 'X']):
              return 'X'
          elif (self.estado_actual[i] == ['O', 'O', 'O']):
              return 'O'

      # Victoria diagonal principal:
      if (self.estado_actual[0][0] != '.' and
          self.estado_actual[0][0] == self.estado_actual[1][1] and
          self.estado_actual[0][0] == self.estado_actual[2][2]):
          return self.estado_actual[0][0]

      # Victoria diagonal secundaria:
      if (self.estado_actual[0][2] != '.' and
          self.estado_actual[0][2] == self.estado_actual[1][1] and
          self.estado_actual[0][2] == self.estado_actual[2][0]):
          return self.estado_actual[0][2]

      # Mesa llena:
      for i in range(0, 3):
          for j in range(0, 3):
              # Si hay espacio, seguimos jugando:
              if (self.estado_actual[i][j] == '.'):
                  return None

      # Empate, mesa llena sin victoria:
      return '.'

    # 'O' es max, en este caso la IA:
    def max(self):

        # Posibles valores para max:
        # -1 - pierde
        #  0 - empata
        #  1 - gana

        # Inicializamos el resultado a -2, peor que lo peor
        maxv = -2

        px = None
        py = None

        resultado = self.es_fin()

        # Si el juego acaba, debemos devolver el resultado:
        if resultado == 'X':
            return (-1, 0, 0)
        elif resultado == 'O':
            return (1, 0, 0)
        elif resultado == '.':
            return (0, 0, 0)

        for i in range(0, 3):
            for j in range(0, 3):
                if self.estado_actual[i][j] == '.':
                    # Esto es una rama del juego:
                    self.estado_actual[i][j] = 'O'
                    (m, min_i, min_j) = self.min()
                    # Fijamos maxv si es necesario:
                    if m > maxv:
                        maxv = m
                        px = i
                        py = j
                    # Vaciamos el tablero
                    self.estado_actual[i][j] = '.'
        return (maxv, px, py)

    # El jugador 'X' es min, un humano en este caso, esta función simula sus
    # movimientos:
    def min(self):

        #Inicio igual que para max:
        minv = 2

        qx = None
        qy = None

        resultado = self.es_fin()

        if resultado == 'X':
            return (-1, 0, 0)
        elif resultado == 'O':
            return (1, 0, 0)
        elif resultado == '.':
            return (0, 0, 0)

        for i in range(0, 3):
            for j in range(0, 3):
                if self.estado_actual[i][j] == '.':
                    self.estado_actual[i][j] = 'X'
                    (m, max_i, max_j) = self.max()
                    if m < minv:
                        minv = m
                        qx = i
                        qy = j
                    self.estado_actual[i][j] = '.'

        return (minv, qx, qy)

    def jugar(self):
      while True:
          self.dibujar_tablero()
          self.resultado = self.es_fin()

          # Mostrar que el juego termino:
          if self.resultado != None:
              if self.resultado == 'X':
                  print('El ganador es X!')
              elif self.resultado == 'O':
                  print('El ganador es O!')
              elif self.resultado == '.':
                  print("Empate!")

              self.iniciar_juego()
              return

          #  Si nos toca jugar:
          if self.turno_jugador == 'X':

              while True:

                  inicio = time.time()
                  (m, qx, qy) = self.min()
                  fin = time.time()
                  print('Evaluacion: {}s'.format(round(fin - inicio, 7)))
                  print('Movimientos Recomendados: X = {}, Y = {}'.format(qx, qy))

                  px = int(input('X coordenada: '))
                  py = int(input('Y coordenada: '))

                  (qx, qy) = (px, py)

                  if self.es_valido(px, py):
                      self.estado_actual[px][py] = 'X'
                      self.turno_jugador = 'O'
                      break
                  else:
                      print('Movimiento no valido, prueba otra vez.')

          # Turno de la IA
          else:
              (m, px, py) = self.max()
              self.estado_actual[px][py] = 'O'
              self.turno_jugador = 'X'

In [47]:
g = Juego()
g.jugar()

.| .| .| 
.| .| .| 
.| .| .| 

Evaluacion: 3.4313362s
Movimientos Recomendados: X = 0, Y = 0
X coordenada: 0
Y coordenada: 0
X| .| .| 
.| .| .| 
.| .| .| 

X| .| .| 
.| O| .| 
.| .| .| 

Evaluacion: 0.0402312s
Movimientos Recomendados: X = 0, Y = 1
X coordenada: 0
Y coordenada: 1
X| X| .| 
.| O| .| 
.| .| .| 

X| X| O| 
.| O| .| 
.| .| .| 

Evaluacion: 0.0013463s
Movimientos Recomendados: X = 2, Y = 0
X coordenada: 2
Y coordenada: 0
X| X| O| 
.| O| .| 
X| .| .| 

X| X| O| 
O| O| .| 
X| .| .| 

Evaluacion: 0.0001934s
Movimientos Recomendados: X = 1, Y = 2
X coordenada: 1
Y coordenada: 2
X| X| O| 
O| O| X| 
X| .| .| 

X| X| O| 
O| O| X| 
X| O| .| 

Evaluacion: 2.84e-05s
Movimientos Recomendados: X = 2, Y = 2
X coordenada: 2
Y coordenada: 2
X| X| O| 
O| O| X| 
X| O| X| 

Empate!


# Ejercicio: ¿Puedes modificar las funciones min y max para podar las ramas innecesarias? ¿Que le pasa al tiempo de procesamiento de la busqueda de la IA?

Modifica las funciones min, max y jugar con los ejemplos siguientes rellenando los espacios:

In [48]:
class Ejercicio(Juego):

  def max_alfa_beta(self, alpha, beta):
        maxv = -2
        px = None
        py = None

        resultado = self.es_fin()

        if resultado == 'X':
            return (-1, 0, 0)
        elif resultado == 'O':
            return (1, 0, 0)
        elif resultado == '.':
            return (0, 0, 0)

        for i in  range(0, 3):
            for j in range(0, 3):
                if self.estado_actual[i][j] == '.':
                    self.estado_actual[i][j] = 'O'
                    (m, min_i, in_j) = self.min_alfa_beta(alfa, beta)
                    if m > maxv:
                        maxv = m
                        px = i
                        py = j
                    self.estado_actual[i][j] = '.'

                    
                    if maxv >= beta:
                        return (maxv, px, py)

                    if maxv > alfa:
                        alfa = maxv

        return (maxv, px, py)

In [49]:
def min_alfa_beta(self, alfa, beta):

        minv = 2

        qx = None
        qy = None

        resultado = self.es_fin()

        if resultado == 'X':
            return (-1, 0, 0)
        elif resultado == 'O':
            return (1, 0, 0)
        elif resultado == '.':
            return (0, 0, 0)

        for i in range(0, 3):
            for j in range(0, 3):
                if self.estado_actual[i][j] == '.':
                    self.estado_actual[i][j] = 'X'
                    (m, max_i, max_j) = self.max_alfa_beta(alfa, beta)
                    if m < minv:
                        minv = m
                        qx = i
                        qy = j
                    self.estado_actual[i][j] = '.'

                    if minv <= alpha:
                        return (minv, px, py)

                    if minv < beta:
                        beta = minv

        return (minv, qx, qy)

In [50]:
  def jugar_alfa_beta(self):
     while True:
        self.dibujar_tablero()
        self.resultado = self.es_fin()

        if self.resultado != None:
            if self.resultado == 'X':
                print('Gana X!')
            elif self.resultado == 'O':
                print('Gana O!')
            elif self.resultado == '.':
                print("Empate")


            self.iniciar_juego()
            return

        if self.turno_jugador == 'X':

            while True:
                inicio = time.time()
                (m, qx, qy) = self.min_alfa_beta(-2, 2)
                fin = time.time()
                print('Tiempo de evaluacion: {}s'.format(round(_fin - inicio, 7)))
                print('Movimiento recomendado: X = {}, Y = {}'.format(qx, qy))

                px = int(input('X coordenada: '))
                py = int(input('Y coordenada: '))

                qx = px
                qy = py

                if self.es_valido(px, py):
                    self.estado_actual[px][py] = 'X'
                    self.turno_jugador = 'O'
                    break
                else:
                    print('Movimiento no valido. Prueba con otro movimiento.')

        else:
            (m, px, py) = self.max_alfa_beta(-2, 2)
            self.estado_actual[px][py] = 'O'
            self.turno_jugador = 'X'

In [51]:
gab = Ejercicio()
gab.jugar()

.| .| .| 
.| .| .| 
.| .| .| 

Evaluacion: 3.4921913s
Movimientos Recomendados: X = 0, Y = 0
X coordenada: 0
Y coordenada: 0
X| .| .| 
.| .| .| 
.| .| .| 

X| .| .| 
.| O| .| 
.| .| .| 

Evaluacion: 0.1207869s
Movimientos Recomendados: X = 0, Y = 1
X coordenada: 0
Y coordenada: 1
X| X| .| 
.| O| .| 
.| .| .| 

X| X| O| 
.| O| .| 
.| .| .| 

Evaluacion: 0.0017824s
Movimientos Recomendados: X = 2, Y = 0
X coordenada: 2
Y coordenada: 0
X| X| O| 
.| O| .| 
X| .| .| 

X| X| O| 
O| O| .| 
X| .| .| 

Evaluacion: 0.0001619s
Movimientos Recomendados: X = 1, Y = 2
X coordenada: 1
Y coordenada: 2
X| X| O| 
O| O| X| 
X| .| .| 

X| X| O| 
O| O| X| 
X| O| .| 

Evaluacion: 5.67e-05s
Movimientos Recomendados: X = 2, Y = 2
X coordenada: 2
Y coordenada: 2
X| X| O| 
O| O| X| 
X| O| X| 

Empate!
